In [5]:
import os
from datasets import load_dataset

In [6]:
dataset = load_dataset("dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192")

Using the latest cached version of the dataset since dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/vant/.cache/huggingface/datasets/dariolopez___justicio-boe-a-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192/default/0.0.0/3a3a46039ebfc36a3554740381bea0820a84b5c2 (last modified on Sat Jun  1 01:46:00 2024).


In [7]:
# Define la función de transformación
def extract_contexts(ejemplo):
    return {'contexts': [d['context'] for d in ejemplo['context_qa']]}

# Aplica la transformación
dataset = dataset.map(extract_contexts)
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'answer', 'context_qa', 'response_groq_llama3_70b_8192', 'contexts'],
        num_rows: 515
    })
})

In [8]:
# Elimina la columna original si es necesario
dataset = dataset.remove_columns(['context_qa'])
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'answer', 'response_groq_llama3_70b_8192', 'contexts'],
        num_rows: 515
    })
})

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.environ.get('GROQ_API_KEY'), 
    # model_name="mixtral-8x7b-32768"
    model_name="llama3-70b-8192",    
)

In [11]:
from ragas.llms.prompt import Prompt
from ragas.metrics import context_relevancy, ContextRelevancy


spanish_prompt_context_relevancy = Prompt(
    name="context_relevancy",
    instruction="""Por favor, extraiga las frases relevantes del contexto proporcionado que sean absolutamente necesarias para responder a la siguiente pregunta. Si no encuentra ninguna frase relevante, o si cree que la pregunta no puede responderse a partir del contexto dado, devuelva la frase «Información insuficiente».  Mientras extrae las frases candidatas, no puede realizar ningún cambio en las frases del contexto proporcionado.""",
    input_keys=["question", "context"],
    output_key="candidate sentences",
    output_type="json",
    language='spanish'
)

context_relevancy = ContextRelevancy(
    context_relevancy_prompt=spanish_prompt_context_relevancy
)

In [12]:
context_relevancy

ContextRelevancy(llm=None, name='context_relevancy', evaluation_mode=<EvaluationMode.qc: 3>, context_relevancy_prompt=Prompt(name='context_relevancy', instruction='Por favor, extraiga las frases relevantes del contexto proporcionado que sean absolutamente necesarias para responder a la siguiente pregunta. Si no encuentra ninguna frase relevante, o si cree que la pregunta no puede responderse a partir del contexto dado, devuelva la frase «Información insuficiente».  Mientras extrae las frases candidatas, no puede realizar ningún cambio en las frases del contexto proporcionado.', output_format_instruction='', examples=[], input_keys=['question', 'context'], output_key='candidate sentences', output_type='json', language='spanish'), show_deprecation_warning=False)

In [13]:
import time
from ragas import evaluate
from datasets import Dataset


scores=[]
for idx, row in enumerate(dataset['train']):
    time.sleep(20)  # groq limits: https://console.groq.com/settings/limits
    row = dataset['train'][idx]
    # Crear un nuevo dataset a partir de la fila seleccionada
    dataset_aux = Dataset.from_dict({key: [value] for key, value in row.items()})
    
    score = evaluate(dataset_aux, metrics=[context_relevancy], llm=llm)
    scores.append(score['context_relevancy'])
    print(f"{idx} - {score['context_relevancy']}")

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

0 - 0.058823529411764705


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

1 - 0.14814814814814814


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

512 - 0.05555555555555555


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

513 - 0.05714285714285714


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

514 - 0.07547169811320754


In [14]:
import numpy as np

np.mean(scores)

0.08610675458042573